In [4]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.003

train_dataset = torchvision.datasets.MNIST(root="../data/",
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root="../data/",
                                          train=False,
                                          transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
    
    def forward(self, x):
        # 초기화 값
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0)) 
        out = self.fc(out[:, -1, :])
        return out
    
model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1/num_epochs}], Step [{i+1/total_step}], Loss: {loss.item():.4f}')
            

Epoch [0.5], Step [99.00166666666667], Loss: 0.5386
Epoch [0.5], Step [199.00166666666667], Loss: 0.2602
Epoch [0.5], Step [299.00166666666667], Loss: 0.2726
Epoch [0.5], Step [399.00166666666667], Loss: 0.2002
Epoch [0.5], Step [499.00166666666667], Loss: 0.2070
Epoch [0.5], Step [599.0016666666667], Loss: 0.0783
Epoch [1.5], Step [99.00166666666667], Loss: 0.0859
Epoch [1.5], Step [199.00166666666667], Loss: 0.1349
Epoch [1.5], Step [299.00166666666667], Loss: 0.0499
Epoch [1.5], Step [399.00166666666667], Loss: 0.1159
Epoch [1.5], Step [499.00166666666667], Loss: 0.1770
Epoch [1.5], Step [599.0016666666667], Loss: 0.1750


In [6]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f"Test Accuracy of the model on the 10000 test images: {(100*correct/total)} %")


Test Accuracy of the model on the 10000 test images: 97.31 %


In [9]:
torch.save(model.state_dict(), 'model.ckpt')